# Tutorial

In [1]:
using OrdinaryDiffEq
using ModelingToolkit
using DataDrivenDiffEq
using LinearAlgebra, Optim
using DiffEqFlux, Flux
using DataFrames

using Random
using Plots
gr()
using CSV
using Pkg

Pkg.status()

┌ Info: DataDrivenDiffEq : OccamNet is available.
└ @ DataDrivenDiffEq /Users/adrocampos/.julia/packages/DataDrivenDiffEq/fivVr/src/DataDrivenDiffEq.jl:168


Status `~/covid19/discovering_missing_terms/Project.toml`
  [336ed68f] CSV v0.10.7
⌃ [2445eb08] DataDrivenDiffEq v0.8.6
  [a93c6f00] DataFrames v1.4.3
  [aae7a2af] DiffEqFlux v1.52.0
  [41bf760c] DiffEqSensitivity v6.79.0
  [0c46a032] DifferentialEquations v7.6.0
  [5789e2e9] FileIO v1.16.0
⌃ [587475ba] Flux v0.13.7
  [7073ff75] IJulia v1.23.3
  [033835bb] JLD2 v0.4.28
⌃ [b2108857] Lux v0.4.34
⌃ [961ee093] ModelingToolkit v8.33.0
⌃ [429524aa] Optim v1.7.3
  [7f7a1694] Optimization v3.9.2
  [36348300] OptimizationOptimJL v0.1.4
  [1dea7af3] OrdinaryDiffEq v6.31.2
⌃ [91a5bcdd] Plots v1.36.1
  [e88e6eb3] Zygote v0.6.49
  [8bb1440f] DelimitedFiles
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
Info Packages marked with ⌃ have new versions available and may be upgradable.


# Our data

In [2]:
data_dir = "/Users/adrocampos/covid19/synth_data/"
regions = ["2", "3", "5", "10", "15", "20", "30"][1]
mobility_type = ["inv_dist", "border", "neighbor"][2]
initially_recovered = false

false

In [3]:
positions = CSV.File(data_dir * "positions_" * regions * "_regions.csv")
positions = DataFrame(positions)

Row,id,N,density,x,y
,Int64,Int64,Float64,Float64,Float64
1,1,2780,308.973,0.246368,0.714476
2,2,1081,154.375,0.315868,3.86025


In [4]:
file = "1"

"1"

In [5]:
csv_reader = CSV.File(data_dir * "SIR_" * regions * "_regions_" * mobility_type * "_" * file * ".csv")
df = DataFrame(csv_reader)

index = range(1,stop=5001,step=50)
df = df[index,:]

X = Matrix(df[:, [:S1, :I1, :R1]])'
t = df.t


101-element Vector{Float64}:
   0.0
   5.0
  10.0
  15.0
  20.0
  25.0
  30.0
  35.0
  40.0
  45.0
  50.0
  55.0
  60.0
   ⋮
 445.0
 450.0
 455.0
 460.0
 465.0
 470.0
 475.0
 480.0
 485.0
 490.0
 495.0
 500.0

In [6]:
tspan=(t[begin], t[end])
tsteps = range(tspan[1], tspan[2], length = size(t)[1])

u0 = X[:,1]

3-element Vector{Float64}:
 2773.0
    7.0
    0.0

### Neural ODE for our data

In [7]:
ann = FastChain(
    FastDense(3, 64, tanh), FastDense(64, 64, tanh), FastDense(64, 1)
)

p = [rand(Float32, 3); initial_params(ann)]

function dudt!(du, u, p, t)

    z = ann(u, p[3:end])

    # S, I, R = u
    # β, γ, N = p[1:3]

    N = 2000

    du[1] = -p[1] * u[1] * u[2] / N - z[1] #dS
    du[2] = p[1] * u[1] * u[2] / N - p[2] * u[2] #dI
    du[3] = p[2] * u[2] #dR
end

prob_UODE = ODEProblem(dudt!, u0, tspan, p) ##prob_neuralode

┌ Warning: FastChain is being deprecated in favor of Lux.jl. Lux.jl uses functions with explicit parameters f(u,p) like FastChain, but is fully featured and documented machine learning library. See the Lux.jl documentation for more details.
└ @ DiffEqFlux /Users/adrocampos/.julia/packages/DiffEqFlux/Em1Aj/src/fast_layers.jl:9


ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 500.0)
u0: 3-element Vector{Float64}:
 2773.0
    7.0
    0.0

In [8]:
## Function to train the network
# Define a predictor
function predict(θ, X=X[:, 1], T=t)
    Array(solve(prob_UODE, Vern7(), u0=X, p=θ,
        tspan=(T[1], T[end]), saveat=T,
        abstol=1e-6, reltol=1e-6,
        sensealg=ForwardDiffSensitivity()
    ))
end


## .- instead of -?
function loss(θ)
    X̂ = predict(θ)
    lo = sum(abs2, X - X̂) #/ size(X, 2) + convert(eltype(θ), 1e-3) * sum(abs2, θ[3:end]) ./ length(θ[3:end])
    return X̂, lo
end

# Container to track the losses
losses = Float32[]

# Callback to show the loss during training
callback(θ, args...) = begin
    l = loss(θ) # Equivalent L2 loss
    push!(losses, l)
    if length(losses) % 5 == 0
        println("Current loss after $(length(losses)) iterations: $(losses[end])")
    end
    false
end

callback (generic function with 1 method)

In [9]:


# # No regularisation right now
# function loss(θ)
#     pred = predict(θ)
#     sum(abs2, noisy_data[2:4,:] .- pred[2:4,:]), pred # + 1e-5*sum(sum.(abs, params(ann)))
# end

# loss(p)

# const losses = []
# callback(θ,l,pred) = begin
#     push!(losses, l)
#     if length(losses)%50==0
#         println(losses[end])
#     end
#     false
# end

res1_uode = DiffEqFlux.sciml_train(loss, p, ADAM(0.01), cb=callback, maxiters = 10)
# res2_uode = DiffEqFlux.sciml_train(loss, res1_uode.minimizer, BFGS(initial_stepnorm=0.01), cb=callback, maxiters = 10000)

┌ Warning: sciml_train is being deprecated in favor of direct usage of Optimization.jl. Please consult the Optimization.jl documentation for more details. Optimization.jl's PolyOpt solver is the polyalgorithm of sciml_train
└ @ DiffEqFlux /Users/adrocampos/.julia/packages/DiffEqFlux/Em1Aj/src/train.jl:6


LoadError: InterruptException:

In [ ]:
## Training -> First shooting / batching to get a rough estimate

In [13]:
z  = 10
println("z ", z)

z 10


In [11]:



# # First train with ADAM for better convergence -> move the parameters into a
# # favourable starting positing for BFGS
# res1 = DiffEqFlux.sciml_train(shooting_loss, p, ADAM(0.1f0), cb=callback, maxiters=100)
# println("Training loss after $(length(losses)) iterations: $(losses[end])")

# # Train with BFGS to achieve partial fit of the data
# res2 = DiffEqFlux.sciml_train(shooting_loss, res1.minimizer, BFGS(initial_stepnorm=0.01f0), cb=callback, maxiters=500)
# println("Training loss after $(length(losses)) iterations: $(losses[end])")
# # Full L2-Loss for full prediction
# res3 = DiffEqFlux.sciml_train(loss, res2.minimizer, BFGS(initial_stepnorm=0.01f0), cb=callback, maxiters=100)
# println("Final training loss after $(length(losses)) iterations: $(losses[end])")

In [ ]:
node_preds = predict_neuralode(result_neuralode2.u)

pS = plot(tsteps, [X[1,:], node_preds[1,:]], label=["data S" "prediction S"])
pI = plot(tsteps, [X[2,:], node_preds[2,:]], label=["data I" "prediction I"])
pR = plot(tsteps, [X[3,:], node_preds[3,:]], label=["data R" "prediction R"])

display(plot(pS, pI, pR, layout = (3,1), size = (800, 800)))

## Universal Differential Equations

In [ ]:

ann = FastChain(FastDense(3, 16, tanh),
    # FastDense(50, 50, tanh),
    FastDense(16, 1))

p = [rand(Float32, 3); initial_params(ann)]

function dudt_(du, u, p, t)
    
    S, I, R = u
    β, γ, N  = p[1:3]

    z  = ann(u, p[3:end])
    dS = -β * S * I/N - z[1]  # susceptible
    dI =  β * S * I/N - γ*I  # infected
    dR =  γ * I

    du[1] = dS
    du[2] = dI
    du[3] = dR

end




# Define the problem
prob_UODE = ODEProblem(dudt_, u0, tspan, p) ##prob_neuralode


In [ ]:
# use Optimization.jl to solve the problem
adtype = Optimization.AutoZygote()
optf    = Optimization.OptimizationFunction((x,p) -> loss(x), adtype)
optprob = Optimization.OptimizationProblem(optf, p)

result_neuralode = Optimization.solve(optprob, ADAM(0.05), tspan=tspan, saveat=t, maxiters = 10)